In [1]:
import numpy as np
import json
import time, os
import math

# Global variables / dataset 
panoptic_path = "../nas/MAEVE/dataset/panoptic-toolbox/"
sequences = ["161029_tools1"] #"170915_office1","160422_ultimatum1","160906_band4","161029_build1","161029_sports1","161029_tools1","170407_haggling_a1"]
sequence_name = "161029_tools1"
out_path = os.path.join(panoptic_path,"trtpose3D/")
sequence_path = os.path.join(panoptic_path,sequence_name)

CONTINUOUS_STATE_PARTS = [
            "nose", "left_ear", "right_ear", "left_shoulder", "right_shoulder", 
            "left_elbow", "right_elbow", "left_wrist", "right_wrist", "left_hip", "right_hip", "left_knee", 
            "right_knee", "left_ankle", "right_ankle", "neck"]

mapping_panoptic_cstate = [1,16,18,3,9,4,10,5,11,6,12,7,13,8,14,0]

In [2]:
# BUILD THE GROUND TRUTH JSON
# Merge a unique json with ground truth
gt = []
directory= os.path.join(sequence_path,"hdPose3d_stage1_coco19")

for filename in sorted(os.listdir(directory)):
    with open(os.path.join(directory,filename),'r') as f:
        data  = json.load(f)
        json_frame = {}
        json_frame["frame_id"] = int(filename.split('_')[1].split('.')[0])
        json_frame["timestamp"] = data["univTime"]
        # print(json_frame["timestamp"])
        json_frame["kp3d"] = []
        json_frame["kp2d"] = []
        json_frame["continuousState"] = []
        json_frame["track_ids"] = []
        for body in data["bodies"]:
            json_frame["track_ids"].append(body["id"])
            kp19 = np.array(body["joints19"]).reshape(-1,4)
            kp18 = kp19[mapping_panoptic_cstate,0:3]/100
            acc = kp19[mapping_panoptic_cstate,3]*100 # Because it's too low for visualization
            json_frame["continuousState"].append(kp18.tolist())
            kp3d = {}
            kp2d = {}
            for i,label in enumerate(CONTINUOUS_STATE_PARTS):
                kp3d[label] = kp18[i,:].tolist()    
                kp2d[label] = [0,0,acc[i]]
            json_frame["kp3d"].append(kp3d)
            json_frame["kp2d"].append(kp2d)
        gt.append(json_frame)

# dump json
with open(os.path.join(out_path,sequence_name+".gt.json"), "w") as outfile:
    json.dump(gt, outfile)

# Export in viewer format
os.system('python3 to_viewer.py '+out_path+sequence_name+'.gt.json --rotation 180 --output "tmp/"'+sequence_name+'".gt.viewer.json" >> "tmp/"$1"_log_2.txt"')

0

In [3]:
# Build the dictionary frame_id:timestamp
time_oracle = {}
for frame in gt:
    time_oracle[frame["frame_id"]] = frame["timestamp"]
time_oracle

{143: 47439.864,
 144: 47473.22,
 145: 47506.599,
 146: 47539.955,
 147: 47573.333,
 148: 47606.712,
 149: 47640.068,
 150: 47673.424,
 151: 47706.803,
 152: 47740.181,
 153: 47773.537,
 154: 47806.893,
 155: 47840.272,
 156: 47873.651,
 157: 47907.007,
 158: 47940.363,
 159: 47973.741,
 160: 48007.12,
 161: 48040.454,
 162: 48073.832,
 163: 48107.188,
 164: 48140.59,
 165: 48173.923,
 166: 48207.302,
 167: 48240.658,
 168: 48274.059,
 169: 48307.392,
 170: 48340.771,
 171: 48374.127,
 172: 48407.528,
 173: 48440.862,
 174: 48474.24,
 175: 48507.596,
 176: 48540.998,
 177: 48574.331,
 178: 48607.71,
 179: 48641.066,
 180: 48674.467,
 181: 48707.8,
 182: 48741.179,
 183: 48774.535,
 184: 48807.914,
 185: 48841.27,
 186: 48874.649,
 187: 48908.005,
 188: 48941.383,
 189: 48974.739,
 190: 49008.118,
 191: 49041.474,
 192: 49074.853,
 193: 49108.209,
 194: 49141.587,
 195: 49174.943,
 196: 49208.322,
 197: 49241.678,
 198: 49275.057,
 199: 49308.413,
 200: 49341.769,
 201: 49375.147,
 202:

In [6]:
# CREATE THE CAMERA JSON FILES
# Merge a unique json for all frames of each camera
cameras = [8] # [6,7,8,9,10]    # 6 to 10 are the upper ones
for camera in cameras:
    cam_data = []
    directory= os.path.join(sequence_path,"trtpose3D","50_"+str(camera).zfill(2))
    print("Doing",directory)
    for filename in sorted(os.listdir(directory)):
        with open(os.path.join(directory,filename),'r') as f:
            data  = json.load(f)
            # print(data)
            json_frame = {}
            json_frame["frame_id"] = data["frame_id"]
            
            # json_frame["timestamp"] = data["timestamp"] # This seems wrong, use the gt instead
            if data["frame_id"] not in time_oracle:
                continue
            json_frame["timestamp"] = time_oracle[data["frame_id"]]
            
            
            json_frame["kp2d"] = []
            json_frame["kp3d"] = []
            json_frame["continuousState"] = []            
            json_frame["track_ids"] = []
            # for body in data["bodies"]:
                # pass
            if data["kp3d"][0] is not None:
                for j,body in enumerate(data["kp3d"]):
                    kp18 = np.empty((len(CONTINUOUS_STATE_PARTS),3))+np.nan
                    kp3d = {}
                    kp2d = {}
                    for i,label in enumerate(CONTINUOUS_STATE_PARTS):
                        if label in body.keys():
                            # print("2d",list(data["kp2d"][j].keys()))
                            # print("3d",list(data["kp3d"][j].keys()))
                            if '_NaN_' not in body[label]:
                                kp3d[label] = (np.array(body[label])/100).tolist()
                                # kp3d[label] = [None if x == '_NaN_' or x == 'NaN' else x/100 for x in body[label]] 
                                kp2d[label] = data["kp2d"][j][label]
                                kp18[i,:] = np.array(kp3d[label]).reshape(1,3) # cm to m

                    if kp3d != {} and kp2d != {} and len(list(kp2d.keys())) > 2: # At least 3 keypoints
                        json_frame["track_ids"].append(camera)
                        json_frame["continuousState"].append([[] if math.isnan(x[0]) else x for x in kp18.tolist()])
                        json_frame["kp3d"].append(kp3d)
                        json_frame["kp2d"].append(kp2d)
            cam_data.append(json_frame)
            # if json_frame["frame_id"] == 1169:
            #     print(os.path.join(directory,filename))
            #     break
        # dump json

    with open(os.path.join(out_path,sequence_name+"."+ str(camera) +".json"), "w") as outfile:
        json.dump(cam_data, outfile)
    # Export in viewer format
    os.system("python3 to_viewer.py "+out_path+sequence_name+"."+ str(camera) +".json --rotation 180 --output tmp/"+sequence_name + "." + str(camera) + ".viewer.json >> tmp/$1_log_2.txt")
    # os.system("python3 to_viewer.py tmp/"+sequence_name+"."+ str(camera) +".json --rotation 180 --output tmp/"+sequence_name + "." + str(camera) + ".viewer.json")

Doing ../nas/MAEVE/dataset/panoptic-toolbox/161029_tools1/trtpose3D/50_06


KeyboardInterrupt: 

In [5]:
os.path.join(out_path,sequence_name+"."+ str(camera) +".json")

'../nas/MAEVE/dataset/panoptic-toolbox/trtpose3D/161029_tools1.8.json'

In [28]:
# # Aggregate cameras
# cameras = [6,7,8,9,10]
# data = {}
# for camera in cameras:
#     with open(os.path.join("tmp/",sequence_name+"."+ str(camera) +".json"), "r") as f:
#         file = json.load(f)
#         for frame in file:
#             if frame["frame_id"] not in data:
#                 data[frame["frame_id"]] = []
#             data[frame["frame_id"]].append(frame)
        
# # Make one big json file
# total_out = []
# for frame_id in sorted(list(data.keys())):
#     total = {}
#     # print(data[frame_id])
#     total["frame_id"] = frame_id
#     total["timestamp"] = data[frame_id][0]["timestamp"]
#     total['kp2d'] = []
#     total['kp3d'] = []
#     total['continuousState'] = []
#     total['track_ids'] = []
#     for c in data[frame_id]:
#         for i in range(len(c["kp2d"])):
#             total["kp2d"].append(c["kp2d"][i])
#             total["kp3d"].append(c["kp3d"][i])
#             total["continuousState"].append(c["continuousState"][i])
#             total["track_ids"].append(c["track_ids"][i])
#     total_out.append(total)
#     # if len(total['kp3d']) > 1:
#     #     print(total)
#     #     break

# filename_out = sequence_name+"."+ ".".join(map(str, cameras)) +".json" 

# with open(os.path.join("tmp/",filename_out), "w") as outfile:
#     json.dump(total_out, outfile)
# # Export in viewer format
# os.system("python3 to_viewer.py tmp/"+ filename_out + " --rotation 180 --output tmp/"+ filename_out.replace('.json','.viewer.json') + " >> tmp/$1_log_2.txt")
    